In [ ]:
import pyscf
from pyscf import lo
import Driver_SCF
import get_atom_orb
import Util_Mole
from functools import reduce
import numpy
import Chem_Bond_Analysis
import seaborn,pandas
import matplotlib.pyplot as plt
import Util_Math
from pyscf.symm.Dmatrix import *

In [ ]:
A = get_euler_angles([1,0,0],[0,1,0])
print(A)

In [ ]:
def get_bas_rotate_matrix(mol, alpha, beta, gamma):
    res = numpy.zeros((mol.nao, mol.nao), dtype=numpy.float64)
    loc = 0
    for i in range(mol.nbas):
        l = mol.bas_angular(i)
        dmat = Dmatrix(l, alpha, beta, gamma, reorder_p=True)
        for _ in range(mol.bas_nctr(i)):
            res[loc:loc+2*l+1, loc:loc+2*l + 1] = dmat
            loc += 2*l+1
    return numpy.matrix(res)


In [ ]:
def get_rotated_mol_coord(mol, rot_center, alpha, beta, gamma):
    coord = mol.atom_coords() * 0.52917720859
    # print(coord)
    coord -= rot_center
    # print(coord)
    rot_mat = Util_Math.get_rotation_matrix_euler_angle_ZYZ(alpha, beta, gamma)
    # print('rot_mat = ', rot_mat)
    # print(coord.T)
    coord = numpy.dot(rot_mat, coord.T).T
    # print('product ', numpy.dot(rot_mat, coord))
    # print('coord', coord)
    coord += rot_center
    # print(coord)
    res = []
    for i in range(mol.natm):
        res.append([mol.atom_symbol(i), coord[i, :]])

    return res

def mocoeff_phase_canonicalize(mo_coeff):
    for i in range(mo_coeff.shape[1]):
        for j in range(mo_coeff.shape[0]):
            if abs(mo_coeff[j,i]) > 1e-4:
                if mo_coeff[j,i] < 0:
                    mo_coeff[:,i] *=-1
                break
    return mo_coeff

In [ ]:
basis = 'ccpvdz'
# mol = Util_Mole.get_mol(xyz = "O 0 0 0\n H 1 0 0\n H 0 1 0",basis="sto-3g")
mol = Util_Mole.get_mol(xyz = "O 0 0 0\n H 0 1 0\n H -1 0 0",basis=basis,print_verbose=4)
rohf = Driver_SCF.Run_SCF(mol)


In [ ]:
for i in range(mol.nbas):
    print('shell %d on atom %d l = %s has %d contracted GTOs' %
          (i, mol.bas_atom(i), mol.bas_angular(i), mol.bas_nctr(i)))
mol.ao_labels()

In [ ]:
xyz_new = get_rotated_mol_coord(mol,[0.0,0.0,0.0],A[0],A[1],A[2])
print(xyz_new)
bas_rot_mat = get_bas_rotate_matrix(mol,A[0],A[1],A[2])
print(mol.basis)
mol_new = Util_Mole.get_mol(xyz = xyz_new,basis=basis,print_verbose=4)
rohf_new = Driver_SCF.Run_SCF(mol_new)
# d 矩阵的顺序 ? 


In [ ]:
for i in range(mol_new.nbas):
    print('shell %d on atom %d l = %s has %d contracted GTOs' %
          (i, mol_new.bas_atom(i), mol_new.bas_angular(i), mol_new.bas_nctr(i)))
mol_new.ao_labels()

In [ ]:
bas_rot_mat[8:13,8:13]

In [ ]:
rohf_mo_coeff_phase = mocoeff_phase_canonicalize(rohf.mo_coeff)
rohf_new_mo_coeff_phase = mocoeff_phase_canonicalize(rohf_new.mo_coeff)
rohf_rot_mo_coeff_phase = mocoeff_phase_canonicalize(numpy.dot(bas_rot_mat,rohf.mo_coeff))

In [ ]:
print(rohf.mo_coeff[9:14,0:2])
print(rohf_new.mo_coeff[9:14,0:2])
print(rohf_rot_mo_coeff_phase[9:14,0:2])
